---


## Data Processing and Analysis - DataScience Course
#### Student: Wanderson Santos

---



---



###Objective: 
Find indivuduals from database B who are also present on database A applying 
Record Linkage techniques based on similarities.

---



---



## Framework Initialization

In [90]:
!wget -q http://archive.apache.org/dist/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
!tar xf spark-2.4.2-bin-hadoop2.7.tgz
!pip install -q findspark


!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install jellyfish

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from google.colab import drive
import pandas as pd
import jellyfish

### Loading data from Google Drive

In [91]:
drive.mount("/content/gDrive", force_remount=True)
!ls

Mounted at /content/gDrive
gDrive			       spark-2.4.2-bin-hadoop2.7.tgz.1
sample_data		       spark-2.4.4-bin-hadoop2.7
spark-2.4.2-bin-hadoop2.7      spark-2.4.4-bin-hadoop2.7.tgz
spark-2.4.2-bin-hadoop2.7.tgz


In [128]:
df_base_a = spark.read.csv(
  'gDrive/My Drive/Colab Notebooks/Trabalho_BigData/base_sintetica_ascii_a.csv', 
  header=True
)
df_base_b = spark.read.csv(
  'gDrive/My Drive/Colab Notebooks/Trabalho_BigData/base_sintetica_ascii_b.csv', 
  header=True
)

### Checking DB structure

In [129]:
df_base_a.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)



In [130]:
df_base_b.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)



### Checking number of users on each database

In [131]:
print (df_base_a.count())
print (df_base_b.count())

540
143


### Databases vizualization

In [132]:
df_base_a.show()

+---+--------------------+---------+-----------------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+-----------+--------------+----+
| id|              bairro|      cep|           cidade|           cpf| data_nasc|               email|            endereco|estado|idade|                 mae|                nome|numero|      signo|tipo_sanguineo|_c15|
+---+--------------------+---------+-----------------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+-----------+--------------+----+
|  0|         Vila Sarney|65090-677|         Sao Luis|097.627.958-49|10/09/1948|vicenteolivermont...|2a Travessa do Ri...|    MA|   71|     Louise Beatriz |Vicente Oliver Mo...|   738|     Virgem|           AB-|null|
|  1|      Vila Sao Jorge|75044-220|         Anapolis|685.096.335-09|24/04/1995|miguelbernardogon...|  Praca Joao Salomao|    GO|   

In [133]:
df_base_b.show()

+---+---------+-----------------+--------------+----------+--------------------+------+--------------------+--------------------+--------------+
| id|      cep|           cidade|           cpf| data_nasc|            endereco|estado|                 mae|                nome|tipo_sanguineo|
+---+---------+-----------------+--------------+----------+--------------------+------+--------------------+--------------------+--------------+
| 01|69304-350|        Boa Vista|506.865.202-46|22/11/1965|Avenida Mario Hom...|    RR|Isabelle Marcia A...|Felipe Bruno dos ...|           AB-|
| 02|78132-360|    Varzea Grande|325.828.867-42|      null|      Rua Rio Cuiaba|    MT|       Alana Isadora|Marcio Heitor Sil...|           AB-|
| 03|88810-336|         Criciuma|284.536.218-88|23/09/1961|Rua Romeu Lopes d...|    SC|  Caroline Gabriela |Theo Mario Victor...|            A-|
| 04|60874-405|        Fortaleza|390.375.925-20|11/05/1981|                null|    CE|       Tereza Bruna |Pietro Henrique d...| 

### Renaming columns

In [134]:
df_base_a = df_base_a.withColumnRenamed('id', 'id_a')
df_base_a = df_base_a.withColumnRenamed('bairro', 'bairro_a')
df_base_a = df_base_a.withColumnRenamed('cep', 'cep_a')
df_base_a = df_base_a.withColumnRenamed('cidade', 'cidade_a')
df_base_a = df_base_a.withColumnRenamed('cpf', 'cpf_a')
df_base_a = df_base_a.withColumnRenamed('data_nasc', 'data_nasc_a')
df_base_a = df_base_a.withColumnRenamed('email', 'email_a')
df_base_a = df_base_a.withColumnRenamed('endereco', 'endereco_a')
df_base_a = df_base_a.withColumnRenamed('estado', 'estado_a')
df_base_a = df_base_a.withColumnRenamed('idade', 'idade_a')
df_base_a = df_base_a.withColumnRenamed('mae', 'mae_a')
df_base_a = df_base_a.withColumnRenamed('nome', 'nome_a')
df_base_a = df_base_a.withColumnRenamed('numero', 'numero_a')
df_base_a = df_base_a.withColumnRenamed('signo', 'signo_a')
df_base_a = df_base_a.withColumnRenamed('tipo_sanguineo', 'tipo_sanguineo_a')
df_base_a = df_base_a.withColumnRenamed('_c15', '_c15_a')

df_base_a.show()

+----+--------------------+---------+-----------------+--------------+-----------+--------------------+--------------------+--------+-------+--------------------+--------------------+--------+-----------+----------------+------+
|id_a|            bairro_a|    cep_a|         cidade_a|         cpf_a|data_nasc_a|             email_a|          endereco_a|estado_a|idade_a|               mae_a|              nome_a|numero_a|    signo_a|tipo_sanguineo_a|_c15_a|
+----+--------------------+---------+-----------------+--------------+-----------+--------------------+--------------------+--------+-------+--------------------+--------------------+--------+-----------+----------------+------+
|   0|         Vila Sarney|65090-677|         Sao Luis|097.627.958-49| 10/09/1948|vicenteolivermont...|2a Travessa do Ri...|      MA|     71|     Louise Beatriz |Vicente Oliver Mo...|     738|     Virgem|             AB-|  null|
|   1|      Vila Sao Jorge|75044-220|         Anapolis|685.096.335-09| 24/04/1995|mi

In [135]:
df_base_b = df_base_b.withColumnRenamed('id', 'id_b')
df_base_b = df_base_b.withColumnRenamed('cep', 'cep_b')
df_base_b = df_base_b.withColumnRenamed('cidade', 'cidade_b')
df_base_b = df_base_b.withColumnRenamed('cpf', 'cpf_b')
df_base_b = df_base_b.withColumnRenamed('data_nasc', 'data_nasc_b')
df_base_b = df_base_b.withColumnRenamed('endereco', 'endereco_b')
df_base_b = df_base_b.withColumnRenamed('estado', 'estado_b')
df_base_b = df_base_b.withColumnRenamed('mae', 'mae_b')
df_base_b = df_base_b.withColumnRenamed('nome', 'nome_b')
df_base_b = df_base_b.withColumnRenamed('tipo_sanguineo', 'tipo_sanguineo_b')

df_base_b.show()

+----+---------+-----------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+
|id_b|    cep_b|         cidade_b|         cpf_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|
+----+---------+-----------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+
|  01|69304-350|        Boa Vista|506.865.202-46| 22/11/1965|Avenida Mario Hom...|      RR|Isabelle Marcia A...|Felipe Bruno dos ...|             AB-|
|  02|78132-360|    Varzea Grande|325.828.867-42|       null|      Rua Rio Cuiaba|      MT|       Alana Isadora|Marcio Heitor Sil...|             AB-|
|  03|88810-336|         Criciuma|284.536.218-88| 23/09/1961|Rua Romeu Lopes d...|      SC|  Caroline Gabriela |Theo Mario Victor...|              A-|
|  04|60874-405|        Fortaleza|390.375.925-20| 11/05/1981|                null|      CE|   

## FIRST EVALUATION METHOD - Jellyfish with Jaro Winkler

### Selecting the attributes which will be compared

In [136]:
df_base_a_method1 = df_base_a.select(['cpf_a', 'data_nasc_a','nome_a', 'mae_a','tipo_sanguineo_a'])
df_base_b_method1 = df_base_b.select(['cpf_b','data_nasc_b', 'nome_b','mae_b', 'tipo_sanguineo_b'])
print (df_base_a_method1.count())
print (df_base_b_method1.count())
print (df_base_a_method1.show())
print (df_base_b_method1.show())

540
143
+--------------+-----------+--------------------+--------------------+----------------+
|         cpf_a|data_nasc_a|              nome_a|               mae_a|tipo_sanguineo_a|
+--------------+-----------+--------------------+--------------------+----------------+
|097.627.958-49| 10/09/1948|Vicente Oliver Mo...|     Louise Beatriz |             AB-|
|685.096.335-09| 24/04/1995|Miguel Bernardo G...|     Raquel Barbara |              A-|
|956.997.968-27| 13/03/1984|Erick Theo Marcos...|Maite Valentina R...|              B+|
|631.875.885-22| 27/09/1965|Mario Roberto Isa...|Alicia Cristiane ...|             AB+|
|397.480.478-14| 24/11/1965|Elias Anderson Ma...|     Tatiane Josefa |              A+|
|216.419.173-00| 27/04/1997|  Gabriel Luis Jesus|Fernanda Sebastia...|              B-|
|148.386.650-59| 10/01/1985|Bruno Julio Guilh...|     Lavinia Isabel |              B-|
|654.325.863-20| 06/12/1957|Theo Guilherme Souza|Andreia Evelyn Ca...|              O+|
|704.817.633-41| 12/11/1

### Generating phonetics

In [137]:
def generateMetaphone(col):
  return jellyfish.metaphone(col)
udf_criaMetaphone = F.udf(generateMetaphone, StringType())

In [138]:
df_base_a_method1 = df_base_a_method1.withColumn('phonetic_nome_a', udf_criaMetaphone(F.col('nome_a')))
df_base_a_method1 = df_base_a_method1.withColumn('phonetic_mae_a', udf_criaMetaphone(F.col('mae_a')))

df_base_b_method1 = df_base_b_method1.withColumn('phonetic_nome_b', udf_criaMetaphone(F.col('nome_b')))
df_base_b_method1 = df_base_b_method1.withColumn('phonetic_mae_b', udf_criaMetaphone(F.col('mae_b')))

### Checking and Removing registers with missing parameters

In [139]:
print(df_base_a_method1.filter((df_base_a_method1['nome_a'].isNull())).count())
print(df_base_b_method1.filter((df_base_b_method1['nome_b'].isNull())).count())

0
2


In [140]:
df_base_b_method1 = df_base_b_method1.select([
  'cpf_b',
  'data_nasc_b',
  'nome_b',
  'mae_b',
  'tipo_sanguineo_b',
  'phonetic_nome_b',
  'phonetic_mae_b'
]).filter((df_base_b_method1['nome_b'].isNotNull()))

print(df_base_b_method1.count())
df_base_b_method1.show()

141
+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+
|         cpf_b|data_nasc_b|              nome_b|               mae_b|tipo_sanguineo_b|     phonetic_nome_b| phonetic_mae_b|
+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+
|506.865.202-46| 22/11/1965|Felipe Bruno dos ...|Isabelle Marcia A...|             AB-|     FLP BRN TS SNTS|ISBL MRX ANTNL |
|325.828.867-42|       null|Marcio Heitor Sil...|       Alana Isadora|             AB-|        MRS HTR SLFR|       ALN ISTR|
|284.536.218-88| 23/09/1961|Theo Mario Victor...|  Caroline Gabriela |              A-|      0 MR FKTR BRBS|     KRLN KBRL |
|390.375.925-20| 11/05/1981|Pietro Henrique d...|       Tereza Bruna |             AB-|       PTR HNRK T PL|       TRS BRN |
|756.005.387-40| 14/12/1941|Kaique Thiago de ...|      Alice Leticia |              A-|          KK 0K T PL|       ALS LT

### Performing CrossJoin between the datasets

In [143]:
dataset_linkage1 = df_base_a_method1.crossJoin(df_base_b_method1)
print (dataset_linkage1.count())
dataset_linkage1.show()

76140
+--------------+-----------+--------------------+---------------+----------------+---------------+--------------+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+
|         cpf_a|data_nasc_a|              nome_a|          mae_a|tipo_sanguineo_a|phonetic_nome_a|phonetic_mae_a|         cpf_b|data_nasc_b|              nome_b|               mae_b|tipo_sanguineo_b|     phonetic_nome_b| phonetic_mae_b|
+--------------+-----------+--------------------+---------------+----------------+---------------+--------------+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+
|097.627.958-49| 10/09/1948|Vicente Oliver Mo...|Louise Beatriz |             AB-| FSNT OLFR MNTR|      LS BTRS |506.865.202-46| 22/11/1965|Felipe Bruno dos ...|Isabelle Marcia A...|             AB-|     FLP BRN TS SNTS|ISBL MRX ANTNL |
|097.627.958-49| 10/09/1948|Vicente Oliver Mo.

******************************************************************************************************************************************************************************************************************************************

### Similarity Calculation (Jellyfish without weighted attributes)

In [144]:
# Alterada a funcao unicode por str, pois estava apresentando erro.
# Funcao para calcular % do jellyfish.hamming_distance, pois o resultante 
# é a quantidade de caracteres diferentes entre duas strings.
def compare(data_nasc_a,mae_a, nome_a, tipo_sanguineo_a, phonetic_mae_a, phonetic_nome_a,
  data_nasc_b, mae_b, nome_b, tipo_sanguineo_b, phonetic_mae_b, phonetic_nome_b
):
  sim = 0
  sim_nominais = 0
  sim_cat = 0
  # Comparando atributos nominais
  sim_nominais = jellyfish.jaro_winkler(str(mae_a), str(mae_b))
  sim_nominais += jellyfish.jaro_winkler(str(nome_a), str(nome_b))
  sim_nominais += jellyfish.jaro_winkler(str(phonetic_mae_a), str(phonetic_mae_b))
  sim_nominais += jellyfish.jaro_winkler(str(phonetic_nome_a), str(phonetic_nome_b))

  # Comparando categorias
  # Hamming é uma distancia que representa a qtde de caracteres diferetes entre 2 strings.
  # Foi calculado o % com relação a string de maior caractere e para saber a similiarade, precisamos
  # encontrar o complemento da medida.
  # sim_cat não é calculado se a "data_nasc" ou "tipo_sanguineo" de uma das bases estiver vazia 
  
  if not((data_nasc_a) is None) and not((data_nasc_b) is None):
    sim_cat = 1 - (jellyfish.hamming_distance(str(data_nasc_a), str(data_nasc_b)))\
    /(max(len(data_nasc_a),len(data_nasc_b)))
  if not((tipo_sanguineo_a) is None) and not((tipo_sanguineo_b) is None):
    sim_cat += 1 - (jellyfish.hamming_distance(str(tipo_sanguineo_a), str(tipo_sanguineo_b)))\
    /(max(len(tipo_sanguineo_a),len(tipo_sanguineo_b)))
  
  # Media aritmetica simples
  sim = str(abs(float(sim_nominais + sim_cat)/6))
  
  return sim
udf_compare = F.udf(compare, StringType())

In [145]:
result_linkage1 = dataset_linkage1.withColumn('similarity', udf_compare(
  F.col('data_nasc_a'), 
  F.col('mae_a'), 
  F.col('nome_a'), 
  F.col('tipo_sanguineo_a'), 
  F.col('phonetic_mae_a'), 
  F.col('phonetic_nome_a'),
  F.col('data_nasc_b'), 
  F.col('mae_b'), 
  F.col('nome_b'), 
  F.col('tipo_sanguineo_b'), 
  F.col('phonetic_mae_b'), 
  F.col('phonetic_nome_b')
))

result_linkage1.show()

+--------------+-----------+--------------------+---------------+----------------+---------------+--------------+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+-------------------+
|         cpf_a|data_nasc_a|              nome_a|          mae_a|tipo_sanguineo_a|phonetic_nome_a|phonetic_mae_a|         cpf_b|data_nasc_b|              nome_b|               mae_b|tipo_sanguineo_b|     phonetic_nome_b| phonetic_mae_b|         similarity|
+--------------+-----------+--------------------+---------------+----------------+---------------+--------------+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+-------------------+
|097.627.958-49| 10/09/1948|Vicente Oliver Mo...|Louise Beatriz |             AB-| FSNT OLFR MNTR|      LS BTRS |506.865.202-46| 22/11/1965|Felipe Bruno dos ...|Isabelle Marcia A...|             AB-|     FLP BRN TS SNTS|ISBL MRX 

In [146]:
result_linkage1.select([
  'nome_a',
  'mae_a',
  'tipo_sanguineo_a',
  'nome_b','mae_b',
  'tipo_sanguineo_b',
  'similarity'
]).show()

result_linkage1.count()

+--------------------+---------------+----------------+--------------------+--------------------+----------------+-------------------+
|              nome_a|          mae_a|tipo_sanguineo_a|              nome_b|               mae_b|tipo_sanguineo_b|         similarity|
+--------------------+---------------+----------------+--------------------+--------------------+----------------+-------------------+
|Vicente Oliver Mo...|Louise Beatriz |             AB-|Felipe Bruno dos ...|Isabelle Marcia A...|             AB-| 0.6141506373390432|
|Vicente Oliver Mo...|Louise Beatriz |             AB-|Marcio Heitor Sil...|       Alana Isadora|             AB-| 0.5585523373566851|
|Vicente Oliver Mo...|Louise Beatriz |             AB-|Theo Mario Victor...|  Caroline Gabriela |              A-| 0.5182246671739424|
|Vicente Oliver Mo...|Louise Beatriz |             AB-|Pietro Henrique d...|       Tereza Bruna |             AB-| 0.6765830988657076|
|Vicente Oliver Mo...|Louise Beatriz |             AB-|

76140

In [147]:
result_linkage1.select([
  'nome_a',
  'mae_a',
  'tipo_sanguineo_a',
  'nome_b','mae_b',
  'tipo_sanguineo_b',
  'similarity'
]).orderBy('similarity', ascending=[0]).show()

+--------------------+--------------------+----------------+--------------------+--------------------+----------------+----------+
|              nome_a|               mae_a|tipo_sanguineo_a|              nome_b|               mae_b|tipo_sanguineo_b|similarity|
+--------------------+--------------------+----------------+--------------------+--------------------+----------------+----------+
|Theo Sebastiao da...|Isadora Lucia Joana |              A-|Theo Sebastiao da...|Isadora Lucia Joana |              A-|       1.0|
|Severino Ruan Cal...|   Juliana Jennifer |              A+|Severino Ruan Cal...|   Juliana Jennifer |              A+|       1.0|
|   Breno Bruno Pinto|     Jessica Tereza |             AB+|   Breno Bruno Pinto|     Jessica Tereza |             AB+|       1.0|
|Jose Caua Vicente...|      Luzia Luciana |              O-|Jose Caua Vicente...|      Luzia Luciana |              O-|       1.0|
|Alexandre Bruno M...|   Isabelly Kamilly |              B-|Alexandre Bruno M...|  

### Excluding duplicates from B in A

In [151]:
# Removing duplicate data from crossjoin. We just want data from dataset B included in dataset A
result_linkage_tratado_b1 = result_linkage1.orderBy(
  'similarity', 
  ascending=[0]
).drop_duplicates(['nome_b'])

print(result_linkage_tratado_b1.count())

result_linkage_tratado_b1.select([
  'nome_a',
  'nome_b',
  'mae_a',
  'mae_b',
  'cpf_a',
  'cpf_b',
  'data_nasc_a',
  'data_nasc_b',
  'tipo_sanguineo_a',
  'tipo_sanguineo_b',
  'similarity'
]).orderBy('similarity', ascending=[0]).limit(50).show()

127
+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+-----------+----------------+----------------+----------+
|              nome_a|              nome_b|               mae_a|               mae_b|         cpf_a|         cpf_b|data_nasc_a|data_nasc_b|tipo_sanguineo_a|tipo_sanguineo_b|similarity|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+-----------+----------------+----------------+----------+
|Leandro Cesar Jor...|Leandro Cesar Jor...|Jessica Jaqueline...|Jessica Jaqueline...|181.410.022-95|181.410.022-95| 05/01/1973| 05/01/1973|              O-|              O-|       1.0|
|Sebastiao Kaue da...|Sebastiao Kaue da...| Aurora Vera Sophia | Aurora Vera Sophia |408.365.694-82|408.365.694-82| 25/03/1951| 25/03/1951|              A-|              A-|       1.0|
|Bento Kaue Gael d...|Bento Kaue Gael d...|Rosa Rafaela Fabi...|Rosa Ra

#### Result shows that:

*   Similarity > 0.7 e < 0.75 --> There are some similar attributes
*   Similarity >=0.75 --> Sometimes the name, and/or mothers name and/or blood type matches, but the ID is different.

On the next strategy, we will weight the attributes to try better results.

In [152]:
result_linkage_tratado_b1=result_linkage_tratado_b1.select([
  'nome_a',
  'nome_b',
  'mae_a',
  'mae_b',
  'cpf_a',
  'cpf_b',
  'data_nasc_a',
  'data_nasc_b',
  'tipo_sanguineo_a',
  'tipo_sanguineo_b',
  'phonetic_nome_a',
  'phonetic_nome_b',
  'phonetic_mae_a',
  'phonetic_mae_b',
  'similarity'
])
M1_results_for_accuracy_calc = result_linkage_tratado_b1.select([
  'cpf_a',
  'cpf_b',
  'similarity'
]).orderBy('similarity', ascending=[0])


## SECOND EVALUATION METHOD - Jellyfish with Jaro Winkler weigthed attributes





### Selecting the attributes which will be compared

In [163]:
df_base_a_analise2 = df_base_a.select(['cpf_a', 'data_nasc_a','nome_a', 'mae_a','tipo_sanguineo_a'])
df_base_b_analise2 = df_base_b.select(['cpf_b','data_nasc_b', 'nome_b','mae_b', 'tipo_sanguineo_b'])
print(df_base_a_analise2.count())
print(df_base_b_analise2.count())
print(df_base_a_analise2.filter((df_base_a_analise2['nome_a'].isNull())).count())
print(df_base_b_analise2.filter((df_base_b_analise2['nome_b'].isNull())).count())

540
143
0
2


In [164]:
df_base_b_analise2 = df_base_b_analise2.fillna({'nome_b':'N/A'})
print(df_base_a_analise2.filter((df_base_a_analise2['nome_a'].isNull())).count())
print(df_base_b_analise2.filter((df_base_b_analise2['nome_b'].isNull())).count())

0
0


In [165]:
df_base_b_analise2.filter(df_base_b_analise2['nome_b'] == 'N/A').show()

+--------------+-----------+------+--------------------+----------------+
|         cpf_b|data_nasc_b|nome_b|               mae_b|tipo_sanguineo_b|
+--------------+-----------+------+--------------------+----------------+
|880.960.378-89| 04/07/1973|   N/A|Sophia Bruna Manu...|             AB-|
|071.707.642-37| 06/06/1949|   N/A|Pietra Tatiane Ra...|              O-|
+--------------+-----------+------+--------------------+----------------+



### Generating phonetics

In [166]:
df_base_a_analise2 = df_base_a_analise2.withColumn('phonetic_nome_a', udf_criaMetaphone(F.col('nome_a')))
df_base_a_analise2 = df_base_a_analise2.withColumn('phonetic_mae_a', udf_criaMetaphone(F.col('mae_a')))

df_base_b_analise2 = df_base_b_analise2.withColumn('phonetic_nome_b', udf_criaMetaphone(F.col('nome_b')))
df_base_b_analise2 = df_base_b_analise2.withColumn('phonetic_mae_b', udf_criaMetaphone(F.col('mae_b')))
df_base_b_analise2.show()


+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+
|         cpf_b|data_nasc_b|              nome_b|               mae_b|tipo_sanguineo_b|     phonetic_nome_b| phonetic_mae_b|
+--------------+-----------+--------------------+--------------------+----------------+--------------------+---------------+
|506.865.202-46| 22/11/1965|Felipe Bruno dos ...|Isabelle Marcia A...|             AB-|     FLP BRN TS SNTS|ISBL MRX ANTNL |
|325.828.867-42|       null|Marcio Heitor Sil...|       Alana Isadora|             AB-|        MRS HTR SLFR|       ALN ISTR|
|284.536.218-88| 23/09/1961|Theo Mario Victor...|  Caroline Gabriela |              A-|      0 MR FKTR BRBS|     KRLN KBRL |
|390.375.925-20| 11/05/1981|Pietro Henrique d...|       Tereza Bruna |             AB-|       PTR HNRK T PL|       TRS BRN |
|756.005.387-40| 14/12/1941|Kaique Thiago de ...|      Alice Leticia |              A-|          KK 0K T PL|       ALS LTX |


In [167]:
df_base_b_analise2.filter(df_base_b_analise2['nome_b'] == 'N/A').show()

+--------------+-----------+------+--------------------+----------------+---------------+--------------+
|         cpf_b|data_nasc_b|nome_b|               mae_b|tipo_sanguineo_b|phonetic_nome_b|phonetic_mae_b|
+--------------+-----------+------+--------------------+----------------+---------------+--------------+
|880.960.378-89| 04/07/1973|   N/A|Sophia Bruna Manu...|             AB-|              N|   SF BRN MNL |
|071.707.642-37| 06/06/1949|   N/A|Pietra Tatiane Ra...|              O-|              N|   PTR TXN RS |
+--------------+-----------+------+--------------------+----------------+---------------+--------------+



### Performing CrossJoint between datasets



In [168]:
dataset_linkage2 = df_base_a_analise2.crossJoin(df_base_b_analise2)
print(dataset_linkage2.count())

77220


### Similarity Calculation (Jellyfish with weigthed attributes)

In [179]:
# Alterada a funcao unicode por str, pois estava apresentando erro.
# Funcao para calcular % do jellyfish.hamming_distance, pois o resultante é a 
# quantidade de caracteres diferentes entre duas strings.
def compare2(data_nasc_a, mae_a, nome_a, tipo_sanguineo_a, phonetic_mae_a, 
  phonetic_nome_a, data_nasc_b, mae_b, nome_b, tipo_sanguineo_b, phonetic_mae_b, 
  phonetic_nome_b
):
  sim = 0
  sim_nom_mae = 0
  sim_nom_nome = 0
  sim_nom_ph_mae = 0
  sim_nom_ph_nome = 0
  sim_cat_dtnasc = 0
  sim_cat_sang = 0
  # Comparando atributos nominais
  sim_nom_mae = jellyfish.jaro_winkler(str(mae_a), str(mae_b))
  sim_nom_nome = jellyfish.jaro_winkler(str(nome_a), str(nome_b))
  sim_nom_ph_mae = jellyfish.jaro_winkler(str(phonetic_mae_a), str(phonetic_mae_b))
  sim_nom_ph_nome = jellyfish.jaro_winkler(str(phonetic_nome_a), str(phonetic_nome_b))

  
  # Comparando categorias
  # Hamming é uma distancia que representa a qtde de caracteres diferetes entre 2 strings.
  # Foi calculado o % com relação a string de maior caractere e para saber a similiarade, precisamos
  # encontrar o complemento da medida.
  # sim_cat não é calculado se a "data_nasc" ou "tipo_sanguineo" de uma das bases estiver vazia 
  
  if not((data_nasc_a) is None) and not((data_nasc_b) is None):
    sim_cat_dtnasc = 1 - (jellyfish.hamming_distance(str(data_nasc_a), str(data_nasc_b))) \
    /(max(len(data_nasc_a),len(data_nasc_b)))
  if not((tipo_sanguineo_a) is None) and not((tipo_sanguineo_b) is None):
    sim_cat_sang = 1 - (jellyfish.hamming_distance(str(tipo_sanguineo_a), str(tipo_sanguineo_b))) \
    /(max(len(tipo_sanguineo_a),len(tipo_sanguineo_b)))
  
  #Weights:
  P_mae = 4
  P_nome = 5
  P_ph_mae = 4
  P_ph_nome = 5
  P_data_nasc = 4
  P_tipo_sang = 3
  P_total = P_mae + P_nome + P_ph_mae + P_ph_nome + P_data_nasc + P_tipo_sang
  
 # Weigth Average
  sim = str(abs(float(
    sim_nom_mae*P_mae + \
    sim_nom_nome*P_nome + \
    sim_nom_ph_mae*P_ph_mae + \
    sim_nom_ph_nome*P_ph_nome + \
    sim_cat_dtnasc*P_data_nasc + \
    sim_cat_sang*P_tipo_sang
  )/P_total))
  
  return sim
udf_compare2 = F.udf(compare2, StringType())

In [170]:
result_linkage2 = dataset_linkage2.withColumn('similarity', udf_compare2(
  F.col('data_nasc_a'), 
  F.col('mae_a'), 
  F.col('nome_a'), 
  F.col('tipo_sanguineo_a'), 
  F.col('phonetic_mae_a'), 
  F.col('phonetic_nome_a'),
  F.col('data_nasc_b'), 
  F.col('mae_b'), 
  F.col('nome_b'), 
  F.col('tipo_sanguineo_b'), 
  F.col('phonetic_mae_b'), 
  F.col('phonetic_nome_b')
))

In [178]:
result_linkage2.select([
  'nome_a',
  'nome_b',
  'mae_a',
  'mae_b',
  'cpf_a',
  'cpf_b',
  'data_nasc_a',
  'data_nasc_b',
  'tipo_sanguineo_a',
  'tipo_sanguineo_b',
  'similarity'
]).orderBy('similarity', ascending=[0]).show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+-----------+----------------+----------------+----------+
|              nome_a|              nome_b|               mae_a|               mae_b|         cpf_a|         cpf_b|data_nasc_a|data_nasc_b|tipo_sanguineo_a|tipo_sanguineo_b|similarity|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+-----------+----------------+----------------+----------+
|Otavio Bernardo d...|Otavio Bernardo d...| Catarina Cristiane | Catarina Cristiane |895.160.103-00|895.160.103-00| 10/07/1945| 10/07/1945|              B+|              B+|       1.0|
|Manoel Julio Edua...|Manoel Julio Edua...|Francisca Aurora ...|Francisca Aurora ...|420.868.827-00|420.868.827-00| 24/09/1970| 24/09/1970|              B-|              B-|       1.0|
|Erick Bryan Julio...|Erick Bryan Julio...|   Regina Alice Ana |   Regina A

### Excluding duplicates from B in A

In [180]:
# Removing duplicate data from crossjoin. We just want data from dataset B included in dataset A
result_linkage_tratado_b2 = result_linkage2.orderBy(
  'similarity', 
  ascending=[0]
).drop_duplicates(['nome_b'])

result_linkage_tratado_b2.select([
  'nome_a',
  'nome_b',
  'mae_a',
  'mae_b',
  'cpf_a',
  'cpf_b',
  'data_nasc_a',
  'data_nasc_b',
  'tipo_sanguineo_a',
  'tipo_sanguineo_b',
  'similarity'
]).orderBy('similarity', ascending=[0]).limit(50).show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+-----------+----------------+----------------+----------+
|              nome_a|              nome_b|               mae_a|               mae_b|         cpf_a|         cpf_b|data_nasc_a|data_nasc_b|tipo_sanguineo_a|tipo_sanguineo_b|similarity|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+-----------+----------------+----------------+----------+
|Leandro Cesar Jor...|Leandro Cesar Jor...|Jessica Jaqueline...|Jessica Jaqueline...|181.410.022-95|181.410.022-95| 05/01/1973| 05/01/1973|              O-|              O-|       1.0|
|Otavio Bernardo d...|Otavio Bernardo d...| Catarina Cristiane | Catarina Cristiane |895.160.103-00|895.160.103-00| 10/07/1945| 10/07/1945|              B+|              B+|       1.0|
|Sebastiao Kaue da...|Sebastiao Kaue da...| Aurora Vera Sophia | Aurora Ver

In [181]:
result_linkage_tratado_b2.describe('similarity').show()

+-------+-------------------+
|summary|         similarity|
+-------+-------------------+
|  count|                128|
|   mean| 0.9359984450823515|
| stddev|0.08315702442565624|
|    min| 0.7377400276052452|
|    max|                1.0|
+-------+-------------------+



In [182]:
result_linkage_tratado_b2=result_linkage_tratado_b2.select([
  'nome_a',
  'nome_b',
  'mae_a',
  'mae_b',
  'cpf_a',
  'cpf_b',
  'data_nasc_a',
  'data_nasc_b',
  'tipo_sanguineo_a',
  'tipo_sanguineo_b',
  'phonetic_nome_a',
  'phonetic_nome_b',
  'phonetic_mae_a',
  'phonetic_mae_b',
  'similarity'
])

M2_results_for_accuracy_calc = result_linkage_tratado_b2.select([
  'cpf_a',
  'cpf_b',
  'similarity'
]).orderBy('similarity', ascending=[0])

## THRID EVALUATION METHOD - FuzzyWuzz

### Performing CrossJoint between datasets

In [183]:
dataset_linkage3 = df_base_a.crossJoin(df_base_b)
dataset_linkage3.show()

+----+-----------+---------+--------+--------------+-----------+--------------------+--------------------+--------+-------+---------------+--------------------+--------+-------+----------------+------+----+---------+-----------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+
|id_a|   bairro_a|    cep_a|cidade_a|         cpf_a|data_nasc_a|             email_a|          endereco_a|estado_a|idade_a|          mae_a|              nome_a|numero_a|signo_a|tipo_sanguineo_a|_c15_a|id_b|    cep_b|         cidade_b|         cpf_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|
+----+-----------+---------+--------+--------------+-----------+--------------------+--------------------+--------+-------+---------------+--------------------+--------+-------+----------------+------+----+---------+-----------------+--------------+-----------+--------------------+--------+-------

### Checking if the IDs are unique
Names could be duplicate, but id dont.

In [184]:
dataset_linkage3.select('id_a').distinct().count()

540

In [185]:
dataset_linkage3.select('id_b').distinct().count()

143

### Checking null in each dataset
Null adjustments will be done in the comparison function.

In [189]:
dataset_linkage3.withColumn(
  'numNulls', 
  sum(dataset_linkage3[col].isNull().cast('int') for col in dataset_linkage3.columns)
).orderBy('numNulls', ascending=[0]).show()

+----+--------------+---------+----------+--------------+-----------+--------------------+--------------------+--------+-------+--------------------+--------------------+--------+-----------+----------------+------+----+---------+------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------+
|id_a|      bairro_a|    cep_a|  cidade_a|         cpf_a|data_nasc_a|             email_a|          endereco_a|estado_a|idade_a|               mae_a|              nome_a|numero_a|    signo_a|tipo_sanguineo_a|_c15_a|id_b|    cep_b|    cidade_b|         cpf_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|numNulls|
+----+--------------+---------+----------+--------------+-----------+--------------------+--------------------+--------+-------+--------------------+--------------------+--------+-----------+----------------+------+----+---------+------------+--------------+----

### Selecting data and re-evaluating the nulls


In [190]:
dataset_linkage3_analise = dataset_linkage3.select([
  'id_a',
  'cpf_a',
  'cep_a',
  'cidade_a',
  'data_nasc_a',
  'endereco_a',
  'estado_a',
  'nome_a',
  'mae_a',
  'tipo_sanguineo_a',
  'id_b',
  'cpf_b',
  'cep_b',
  'cidade_b',
  'data_nasc_b',
  'endereco_b',
  'estado_b',
  'nome_b',
  'mae_b',
  'tipo_sanguineo_b'
])

dataset_linkage3_analise.withColumn(
  'numNulls', 
  sum(dataset_linkage3_analise[col].isNull().cast('int') for col in dataset_linkage3_analise.columns)
).orderBy('numNulls', ascending=[0]).show()

+----+--------------+---------+----------+-----------+--------------------+--------+--------------------+--------------------+----------------+----+--------------+---------+------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------+
|id_a|         cpf_a|    cep_a|  cidade_a|data_nasc_a|          endereco_a|estado_a|              nome_a|               mae_a|tipo_sanguineo_a|id_b|         cpf_b|    cep_b|    cidade_b|data_nasc_b|          endereco_b|estado_b|              nome_b|               mae_b|tipo_sanguineo_b|numNulls|
+----+--------------+---------+----------+-----------+--------------------+--------+--------------------+--------------------+----------------+----+--------------+---------+------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------+
|   1|685.096.335-09|75044-220|  Anapolis| 24/04/1995|  Praca Joao Salomao|      GO|Miguel Bernardo G...|    

### Installing Fuzzywuzzy e Levenshtein

In [191]:
!pip install fuzzywuzzy  
!pip install python-Levenshtein  

     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149863 sha256=df552f94b37101c8e3ac2a2bfe0cdf4bf97bf75b300b8949ab98afa08f686a07
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [192]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

### Similarity Calculation Function

In [210]:
def compare3(cep_a,cidade_a,data_nasc_a,endereco_a,estado_a,nome_a,mae_a,tipo_sanguineo_a,
  cep_b,cidade_b,data_nasc_b,endereco_b,estado_b,nome_b,mae_b,tipo_sanguineo_b
):
  
  total=0
  sim = 0
  sim_cep = 0
  sim_cidade = 0
  sim_data_nasc = 0    
  sim_endereco = 0
  sim_estado = 0
  sim_nome = 0
  sim_mae = 0
  sim_tipo_sanguineo = 0
    
  #Calcula similaridade somente para os campos existentes
  if not((cep_a) is None) and not((cep_b) is None):
    sim_cep = fuzz.ratio(cep_a, cep_b)
    total = total + 1
  if not((cidade_a) is None) and not((cidade_b) is None):
    sim_cidade = fuzz.ratio(cidade_a, cidade_b)
    total = total + 1
  if not((data_nasc_a) is None) and not((data_nasc_b) is None):
    sim_data_nasc = fuzz.ratio(data_nasc_a, data_nasc_b)
    total = total + 1
  if not((endereco_a) is None) and not((endereco_b) is None):
    sim_endereco = fuzz.ratio(endereco_a, endereco_b)
    total = total + 1
  if not((estado_a) is None) and not((estado_b) is None):
    sim_estado = fuzz.ratio(estado_a, estado_b)
    total = total + 1
  if not((nome_a) is None) and not((nome_b) is None):
    sim_nome = fuzz.ratio(nome_a, nome_b)
    total = total + 1
  if not((mae_a) is None) and not((mae_b) is None):
    sim_mae = fuzz.ratio(mae_a, mae_b)
    total = total + 1
  if not((tipo_sanguineo_a) is None) and not((tipo_sanguineo_b) is None):
    sim_tipo_sanguineo = fuzz.ratio(tipo_sanguineo_a, tipo_sanguineo_b)
    total = total + 1
      
  #Pesos:
  #P_mae = 4
  #P_nome = 5
  #P_ph_mae = 4
  #P_ph_nome = 5
  #P_data_nasc = 4
  #P_tipo_sang = 3
  #P_total = P_mae + P_nome + P_ph_mae + P_ph_nome + P_data_nasc + P_tipo_sang
  
  # Media simples
  sim = float(
    sim_cep + \
    sim_cidade + \
    sim_data_nasc + \
    sim_endereco + \
    sim_estado + \
    sim_nome + \
    sim_mae + \
    sim_tipo_sanguineo \
  )/total
  
  return sim/100
udf_compare3 = F.udf(compare3, FloatType())

In [211]:
result_linkage3 = dataset_linkage3_analise.withColumn('similarity', udf_compare3(
  F.col('cep_a'), 
  F.col('cidade_a'), 
  F.col('data_nasc_a'), 
  F.col('endereco_a'), 
  F.col('estado_a'), 
  F.col('nome_a'), 
  F.col('mae_a'), 
  F.col('tipo_sanguineo_a'),
  F.col('cep_b'), 
  F.col('cidade_b'), 
  F.col('data_nasc_b'), 
  F.col('endereco_b'), 
  F.col('estado_b'), 
  F.col('nome_b'), 
  F.col('mae_b'), 
  F.col('tipo_sanguineo_b') 
))

In [212]:
result_linkage3_result = result_linkage3.orderBy(['id_b','similarity'], ascending=[0,0])
result_linkage3_result = result_linkage3_result.drop_duplicates(['id_b'])
result_linkage3_result.show()

+----+--------------+---------+--------------------+-----------+--------------------+--------+--------------------+--------------------+----------------+----+--------------+---------+--------------------+-----------+--------------------+--------+--------------------+--------------------+----------------+----------+
|id_a|         cpf_a|    cep_a|            cidade_a|data_nasc_a|          endereco_a|estado_a|              nome_a|               mae_a|tipo_sanguineo_a|id_b|         cpf_b|    cep_b|            cidade_b|data_nasc_b|          endereco_b|estado_b|              nome_b|               mae_b|tipo_sanguineo_b|similarity|
+----+--------------+---------+--------------------+-----------+--------------------+--------+--------------------+--------------------+----------------+----+--------------+---------+--------------------+-----------+--------------------+--------+--------------------+--------------------+----------------+----------+
|  20|756.048.583-90|84172-370|              Cast

In [213]:
M3_results_for_accuracy_calc = result_linkage3_result.select(['cpf_a', 'cpf_b','similarity']).orderBy('similarity', ascending=[0])



---



---



---



---



# FINAL RESULTS EVALUATION

In [214]:
# Let us consider a cuttoff point set as 0.85
cutoff = 0.85

# sorting and deduplicating the resulting dataset
#data = data.withColumn('similairty', F.col('similarity').cast(DoubleType()))
M1_accuracy_calc = M1_results_for_accuracy_calc.orderBy('similarity').dropDuplicates(['cpf_b'])
M1_accuracy_calc = M1_results_for_accuracy_calc.withColumn('match', F.when(F.col('similarity') >= cutoff, '1').otherwise('0'))
M2_accuracy_calc = M2_results_for_accuracy_calc.orderBy('similarity').dropDuplicates(['cpf_b'])
M2_accuracy_calc = M2_results_for_accuracy_calc.withColumn('match', F.when(F.col('similarity') >= cutoff, '1').otherwise('0'))
M3_accuracy_calc = M3_results_for_accuracy_calc.orderBy('similarity').dropDuplicates(['cpf_b'])
M3_accuracy_calc = M3_results_for_accuracy_calc.withColumn('match', F.when(F.col('similarity') >= cutoff, '1').otherwise('0'))

In [215]:
def inspect_pairs(cpf_a, cpf_b, match):
  if match == '1':
    if cpf_a == cpf_b:
      return "TP"
    else:
      return "FP"
  else:
    if cpf_a != cpf_b:
      return "TN"
    else: 
      return "FN"
udf_inspect_pairs = F.udf(inspect_pairs, StringType())

In [216]:
M1_accuracy_calc = M1_accuracy_calc.withColumn('perf', udf_inspect_pairs(
  F.col('cpf_a'), 
  F.col('cpf_b'), 
  F.col('match')
))
M2_accuracy_calc = M2_accuracy_calc.withColumn('perf', udf_inspect_pairs(
  F.col('cpf_a'), 
  F.col('cpf_b'), 
  F.col('match')
))
M3_accuracy_calc = M3_accuracy_calc.withColumn('perf', udf_inspect_pairs(
  F.col('cpf_a'), 
  F.col('cpf_b'), 
  F.col('match')
))
print(M1_accuracy_calc.show())
print(M2_accuracy_calc.show())
print(M3_accuracy_calc.show())

+--------------+--------------+------------------+-----+----+
|         cpf_a|         cpf_b|        similarity|match|perf|
+--------------+--------------+------------------+-----+----+
|081.783.955-09|081.783.955-09|               1.0|    1|  TP|
|488.380.897-10|488.380.897-10|               1.0|    1|  TP|
|475.612.908-02|475.612.908-02|0.9974358974358974|    1|  TP|
|625.938.432-76|625.938.432-76|0.9874458874458875|    1|  TP|
|420.868.827-00|420.868.827-00|               1.0|    1|  TP|
|390.371.546-89|390.371.546-89|               1.0|    1|  TP|
|249.678.124-57|249.678.124-57|               1.0|    1|  TP|
|055.302.287-39|055.302.287-39|0.9947738252086079|    1|  TP|
|506.865.202-46|506.865.202-46|               1.0|    1|  TP|
|327.630.596-86|327.630.596-86|               1.0|    1|  TP|
|658.224.088-93|535.871.894-04|0.7666666666666666|    0|  TN|
|248.115.138-00|248.115.138-00|               1.0|    1|  TP|
|617.496.046-39|442.694.244-60|0.7833333333333333|    0|  TN|
|455.643

## Measures
Accuracy = (TP + TN) / (FP + TP + FN + TN)

Positive predictive value (PPV) = TP / (TP + FP)

Negative predictive value (NPV) = TN / (TN + FN)

Sensitivity (True positive rate) = TP / (TP + FN)

Specificity (True negative rate) = TN / (TN + FP)

In [204]:
dic_results1 = {}
TP = M1_accuracy_calc.filter(F.col('perf') == "TP").count()
TN = M1_accuracy_calc.filter(F.col('perf') == "TN").count()
FP = M1_accuracy_calc.filter(F.col('perf') == "FP").count()
FN = M1_accuracy_calc.filter(F.col('perf') == "FN").count()

dic_results1['accuracy'] =  float(TP + TN) / (FP + TP + FN + TN)
dic_results1['ppv'] = float(TP) / (TP + FP)
dic_results1['npv'] = float(TN) / (TN + FN)
dic_results1['sens'] = float(TP) / (TP + FN)
dic_results1['spec'] = float(TN) / (TN + FP)

In [205]:
dic_results2 = {}
TP = M2_accuracy_calc.filter(F.col('perf') == "TP").count()
TN = M2_accuracy_calc.filter(F.col('perf') == "TN").count()
FP = M2_accuracy_calc.filter(F.col('perf') == "FP").count()
FN = M2_accuracy_calc.filter(F.col('perf') == "FN").count()

dic_results2['accuracy'] =  float(TP + TN) / (FP + TP + FN + TN)
dic_results2['ppv'] = float(TP) / (TP + FP)
dic_results2['npv'] = float(TN) / (TN + FN)
dic_results2['sens'] = float(TP) / (TP + FN)
dic_results2['spec'] = float(TN) / (TN + FP)

In [217]:
dic_results3 = {}
TP = M3_accuracy_calc.filter(F.col('perf') == "TP").count()
TN = M3_accuracy_calc.filter(F.col('perf') == "TN").count()
FP = M3_accuracy_calc.filter(F.col('perf') == "FP").count()
FN = M3_accuracy_calc.filter(F.col('perf') == "FN").count()

dic_results3['accuracy'] =  float(TP + TN) / (FP + TP + FN + TN)
dic_results3['ppv'] = float(TP) / (TP + FP)
dic_results3['npv'] = float(TN) / (TN + FN)
dic_results3['sens'] = float(TP) / (TP + FN)
dic_results3['spec'] = float(TN) / (TN + FP)

In [222]:
final_results1 = pd.DataFrame(dic_results1, index=[0])
final_results2 = pd.DataFrame(dic_results2, index=[0])
final_results3 = pd.DataFrame(dic_results3, index=[0])
print(final_results1)
print(final_results2)
print(final_results3)

   accuracy       ppv       npv      sens  spec
0  0.833333  0.922078  0.648649  0.845238   0.8
   accuracy      ppv       npv      sens      spec
0   0.86087  0.88764  0.769231  0.929412  0.666667
   accuracy  ppv       npv      sens  spec
0  0.979021  1.0  0.933333  0.970297   1.0


******************************************************************************************************************************************************************************************************************************************

******************************************************************************************************************************************************************************************************************************************